## horseIDとjockeyIDの取得

In [6]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
from tqdm import tqdm

In [385]:


race_id_list = df.index.unique()
type(race_id_list[0])
# race_id_list = ["202101010101","202101010102"]

str

In [386]:
test = {}
def scrape_race_results(race_id_lists,par_race_results={}):
  race_results = par_race_results
  for _race_id in tqdm(race_id_lists):
    # if _race_id in race_result.keys():
    #   continue
    time.sleep(1)
    try:
      # スクレイピング
      url = "https://race.netkeiba.com/race/result.html?race_id=" + _race_id
      html = requests.get(url)
      html.encoding= "EUC_JP"
      soup = BeautifulSoup(html.text, "html.parser")
      
      # horse_id
      horse_id_list = []      
      horse_a_list = soup.find('table',attrs={"summary":"全着順"}).find_all('a',attrs={"href":re.compile("/horse/")})
      for a in horse_a_list:
        horse_id = re.findall(r'\d+',a["href"])
        horse_id_list.append(horse_id[0])
      Series1 = pd.Series(data = horse_id_list,dtype=str,name="horse_id")
      # jockey_id
      jockey_id_list = []
      jockey_a_list = soup.find('table',attrs={"summary":"全着順"}).find_all('a',attrs={"href":re.compile("/jockey/")})
      for u in jockey_a_list:
        jockey_id = re.findall(r'\d+',u["href"])
        jockey_id_list.append(jockey_id[0])
      Series2 = pd.Series(data = jockey_id_list,dtype=str,name="jockey_id")
      
      results = pd.concat([Series1, Series2], axis=1)
      test[_race_id] = results
      
    except UnicodeDecodeError as e:
      # print(e)
      # print(f"レースID: {race_id} のデータはありません")
      continue
    except KeyboardInterrupt:
      break
    except TimeoutError:
      break
  return test

In [387]:
results_new = scrape_race_results(race_id_list)
results_new

100%|██████████| 36/36 [00:49<00:00,  1.37s/it]


{'202101010101':       horse_id jockey_id
 0   2018105875     01181
 1   2018105258     01140
 2   2018101134     01096
 3   2018102791     01157
 4   2018101539     01015
 5   2018100570     00666
 6   2018102192     01170
 7   2018102626     01095
 8   2018103582     01180
 9   2018101518     01173
 10  2018106100     01192
 11  2018101447     01091
 12  2018109106     01112
 13  2018106896     01156
 14  2018104780     01144
 15  2018100737     05203,
 '202101010102':       horse_id jockey_id
 0   2018105205     00666
 1   2018100974     01161
 2   2018104870     01182
 3   2018106453     01096
 4   2018100926     01170
 5   2018102052     01157
 6   2018103196     01025
 7   2018101898     01187
 8   2018102542     01192
 9   2018103594     01124
 10  2018106609     01185
 11  2018100435     01166
 12  2018105770     01172
 13  2018100706     01140,
 '202101010103':       horse_id jockey_id
 0   2018100403     01157
 1   2018105166     01182
 2   2018102691     01091
 3   201810070

In [395]:
for key in results_new.keys():
  results_new[key].index = [key] * len(results_new[key])
results = pd.concat([results_new[key] for key in results_new.keys()],sort=False)

In [407]:
print(type(results))
print(len(results))
print(type(df))
print(len(df))

<class 'pandas.core.frame.DataFrame'>
468
<class 'pandas.core.frame.DataFrame'>
468


In [408]:
pd.set_option('display.max_rows',None)

df_add_id = pd.concat([df,results],axis=1)

In [410]:
# df_add_id

In [411]:
# df_add_id.to_pickle("df_add_id.pickle")

----

## idのリストを作成

In [701]:
horse_id_list = df_add_id["horse_id"].unique()
len(horse_id_list)
horse_id_list = horse_id_list
horse_id_list = ['2018105875', '2018105258', '2018101134', '2018102791']

In [638]:
# url = "https://db.netkeiba.com/horse/2019109094"
# url = "https://db.netkeiba.com/horse/2018102444"
# time.sleep(1)
# pd.read_html(url,match='日付')

In [713]:
test = {}
def scrape_horse_results(horse_id_list,par_horse_results={}):
  horse_results = par_horse_results
  for _horse_id in tqdm(horse_id_list):
    # print(_horse_id)
    # if _horse_id in horse_id_list.keys():
    #   continue
    time.sleep(2)
    try:
      # スクレイピング
      url = "https://db.netkeiba.com/horse/" + _horse_id
      time.sleep(1)
      # [0]をつけてリストの中だけを取得
      horse_results[_horse_id] = pd.read_html(url,match='日付')[0]
      
    except UnicodeDecodeError as e:
      # print(e)
      # print(f"レースID: {race_id} のデータはありません")
      continue
    except KeyboardInterrupt:
      break
        
    except TimeoutError:
      break
    # print(horse_results)
      
  return horse_results

In [714]:
horse_results = scrape_horse_results(horse_id_list)

100%|██████████| 4/4 [00:13<00:00,  3.38s/it]


In [715]:
horse_results

{'2018105875':            日付    開催  天気   R          レース名  映像  頭数  枠番  馬番   オッズ  ...   着差  \
 0  2022/01/16  1小倉2   曇  10   小郡特別(2勝クラス) NaN  12   7  10   3.5  ...  1.6   
 1  2021/12/26  6阪神8   曇  12  猪名川特別(2勝クラス) NaN  15   2   2   8.2  ...  0.2   
 2  2021/10/30  4阪神7   晴   8     3歳以上2勝クラス NaN  11   2   2   5.8  ...  0.7   
 3  2021/10/09  4東京1   曇  12     3歳以上2勝クラス NaN  13   5   7  13.8  ...  0.4   
 4  2021/07/04  1函館2   曇   9     3歳以上1勝クラス NaN  16   3   5   7.3  ... -0.4   
 5  2021/06/12  1札幌1  小雨   1         3歳未勝利 NaN  16   5   9  13.5  ... -0.1   
 6  2021/04/10  1新潟1   晴   5         3歳未勝利 NaN  16   8  16  13.7  ...  0.6   
 
    ﾀｲﾑ指数   通過        ペース    上り       馬体重 厩舎ｺﾒﾝﾄ  備考     勝ち馬(2着馬)     賞金  
 0     **  5-6  32.8-35.4  36.5   474(+2)    NaN NaN       メメントモリ    NaN  
 1     **  2-3  35.3-34.8  34.8   472(-6)    NaN NaN    エルカスティージョ    NaN  
 2     **  3-4  35.1-34.9  35.3   478(+2)    NaN NaN       セウラサーリ    NaN  
 3     **  5-8  36.3-34.1  33.9   476(+2)    NaN NaN        

In [717]:
for key in horse_results.keys():
  horse_results[key].index = [key] * len(horse_results[key])
results = pd.concat([horse_results[key] for key in horse_results.keys()],sort=False)

In [719]:
pd.set_option('display.max_rows',10)
results

,日付,開催,天気,R,レース名,映像,頭数,枠番,馬番,オッズ,...,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39
2018105875,2022/01/16,1小倉2,曇,10,小郡特別(2勝クラス),NaN,12,7,10,3.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018105875,2021/12/26,6阪神8,曇,12,猪名川特別(2勝クラス),NaN,15,2,2,8.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018105875,2021/10/30,4阪神7,晴,8,3歳以上2勝クラス,NaN,11,2,2,5.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018105875,2021/10/09,4東京1,曇,12,3歳以上2勝クラス,NaN,13,5,7,13.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018105875,2021/07/04,1函館2,曇,9,3歳以上1勝クラス,NaN,16,3,5,7.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018102791,2020/11/14,3福島5,晴,4,2歳未勝利,NaN,16,2,4,4.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018102791,2020/10/24,4東京5,晴,3,2歳未勝利,NaN,16,4,8,8.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018102791,2020/08/23,3新潟4,晴,2,2歳未勝利,NaN,18,5,9,26.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018102791,2020/08/01,2新潟3,曇,2,2歳未勝利,NaN,17,2,3,48.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [720]:
results.to_csv("horse_vs_data.csv")

In [9]:
df = pd.read_csv('horse_vs_data.csv')
df.iloc[:,:28]


,日付,開催,天気,R,レース名,映像,頭数,枠番,馬番,オッズ,...,着差,ﾀｲﾑ指数,通過,ペース,上り,馬体重,厩舎ｺﾒﾝﾄ,備考,勝ち馬(2着馬),賞金
2018105875,2022/01/16,1小倉2,曇,10,小郡特別(2勝クラス),NaN,12,7,10,3.5,...,1.6,**,5-6,32.8-35.4,36.5,474(+2),NaN,NaN,メメントモリ,NaN
2018105875,2021/12/26,6阪神8,曇,12,猪名川特別(2勝クラス),NaN,15,2,2,8.2,...,0.2,**,2-3,35.3-34.8,34.8,472(-6),NaN,NaN,エルカスティージョ,NaN
2018105875,2021/10/30,4阪神7,晴,8,3歳以上2勝クラス,NaN,11,2,2,5.8,...,0.7,**,3-4,35.1-34.9,35.3,478(+2),NaN,NaN,セウラサーリ,NaN
2018105875,2021/10/09,4東京1,曇,12,3歳以上2勝クラス,NaN,13,5,7,13.8,...,0.4,**,5-8,36.3-34.1,33.9,476(+2),NaN,NaN,レガトゥス,170.0
2018105875,2021/07/04,1函館2,曇,9,3歳以上1勝クラス,NaN,16,3,5,7.3,...,-0.4,**,5-4,33.2-35.2,34.7,474(0),NaN,NaN,(メイショウハボタン),760.0
2018105875,2021/06/12,1札幌1,小雨,1,3歳未勝利,NaN,16,5,9,13.5,...,-0.1,**,4-4,33.8-34.9,34.6,474(-22),NaN,NaN,(バンベルク),510.0
2018105875,2021/04/10,1新潟1,晴,5,3歳未勝利,NaN,16,8,16,13.7,...,0.6,**,3-3,34.9-34.9,35.2,496(0),NaN,NaN,プラチナムレイアー,NaN
2018105258,2022/01/09,1中京3,晴,8,4歳以上1勝クラス,NaN,13,7,10,53.3,...,0.8,**,6-6-6-6,36.6-34.4,34.6,464(+3),NaN,NaN,スタッドリー,NaN
2018105258,2021/11/30,園田,曇,12,C1,NaN,12,1,1,1.6,...,0.0,**,7-6-6-4,0.0-39.7,39.2,461(+7),NaN,NaN,(スマイルヴィヴァン),90.0
2018105258,2021/10/27,園田,晴,9,C1二,NaN,12,6,8,2.1,...,0.7,**,2-2-1-1,0.0-39.9,40.6,454(-7),NaN,NaN,メイショウシマト,36.0
